# Clasificación usando TF-IDF con algoritmos de clasificación

In [1]:
import os
import re
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Leer el archivo Parquet en un DataFrame
df = pd.read_parquet('./eda/mapped_&_usecases_df.parquet')

In [3]:
# Crear una copia de 100,000 filas seleccionadas aleatoriamente
df_sample = df.sample(n=50000, random_state=42).copy()

# Verificar la forma del nuevo DataFrame
print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame sampleado: {df_sample.shape}")

Tamaño del DataFrame original: (115852, 7)
Tamaño del DataFrame sampleado: (50000, 7)


In [4]:
# 3. Dividir los datos en conjunto de entrenamiento y prueba
# Establecer una semilla para la reproducibilidad
seed = 42
np.random.seed(seed)

# Dividir el DataFrame en dos partes iguales
df1, df2 = train_test_split(df, test_size=0.3, random_state=42)

# Verificar las dimensiones de las divisiones
print(f"Size of df1: {df1.shape}")
print(f"Size of df2: {df2.shape}")

print(df2.columns)

# Definir el tamaño de las particiones
train_size = 0.7  # Porcentaje para el conjunto de entrenamiento
val_size = 0.15   # Porcentaje para el conjunto de validación
test_size = 0.15  # Porcentaje para el conjunto de prueba

# Dividir el DataFrame en entrenamiento + validación y prueba
train_val_df, test_df = train_test_split(df2, test_size=test_size + val_size, random_state=42)

# Dividir el DataFrame de entrenamiento + validación en entrenamiento y validación
val_size_adjusted = val_size / (val_size + test_size)  # Ajuste para la proporción en el conjunto de entrenamiento/validación
train_df, val_df = train_test_split(train_val_df, test_size=val_size_adjusted, random_state=42)
               
# Verificar tamaños de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {train_df.shape}")
print(f"Tamaño del conjunto de validación: {val_df.shape}")
print(f"Tamaño del conjunto de prueba: {test_df.shape}")

# Definir el número de muestras que deseas seleccionar
num_train_samples = 12000
num_val_samples = 5000
num_test_samples = 500

# Muestrear X muestras directamente desde los DataFrames
train_df_sampled = train_df.sample(n=num_train_samples, replace=False, random_state=42)
val_df_sampled = val_df.sample(n=num_val_samples, replace=False, random_state=42)
test_df_sampled = test_df.sample(n=num_test_samples, replace=False, random_state=42)

# Verificar tamaños de los conjuntos
print(f"Tamaño del conjunto de entrenamiento sampleado: {train_df_sampled.shape}")
print(f"Tamaño del conjunto de validación sampleado: {val_df_sampled.shape}")
print(f"Tamaño del conjunto de prueba sampleado: {test_df_sampled.shape}")

Size of df1: (81096, 7)
Size of df2: (34756, 7)
Index(['source_code', 'slither_text', 'slither_label', 'use_cases',
       'vulnerability_mapping', 'vulnerability_keys', 'vulnerable'],
      dtype='object')
Tamaño del conjunto de entrenamiento: (12164, 7)
Tamaño del conjunto de validación: (12165, 7)
Tamaño del conjunto de prueba: (10427, 7)
Tamaño del conjunto de entrenamiento sampleado: (12000, 7)
Tamaño del conjunto de validación sampleado: (5000, 7)
Tamaño del conjunto de prueba sampleado: (500, 7)


In [5]:
# 1. Tokenización del código
def tokenize_code(code):
    # Expresión regular mejorada para capturar tokens específicos de Solidity
    tokens = re.findall(r'\b\w+\b|[{}()\[\];,=+\-*/<>!&|%^~]', code)
    return ' '.join(tokens)

# Aplicar tokenización
train_df_sampled['tokenized_code'] = train_df_sampled['source_code'].apply(tokenize_code)
test_df_sampled['tokenized_code'] = test_df_sampled['source_code'].apply(tokenize_code)

# 2. Preprocesar las etiquetas multiclase y multietiqueta
# Dividir las etiquetas por ';' para convertirlas en una lista de etiquetas
train_df_sampled['vulnerability_labels'] = train_df_sampled['vulnerability_mapping'].apply(lambda x: x.split(';'))
test_df_sampled['vulnerability_labels'] = test_df_sampled['vulnerability_mapping'].apply(lambda x: x.split(';'))

# Convertir las etiquetas de texto en un formato binarizado utilizando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df['vulnerability_labels'] = df['vulnerability_mapping'].apply(lambda x: x.split('; '))
mlb.fit(df['vulnerability_labels'])

# Transformar las etiquetas de los conjuntos de entrenamiento y prueba
y_train_sampled = mlb.transform(train_df_sampled['vulnerability_labels'])
y_test_sampled = mlb.transform(test_df_sampled['vulnerability_labels'])

# Separar las características (X) y las etiquetas binarizadas (y)
X_train_sampled = train_df_sampled['tokenized_code']
X_test_sampled = test_df_sampled['tokenized_code']

print(f"Tamaño del conjunto y_train_sampled: {y_train_sampled.shape}")
print(f"Tamaño del conjunto y_test_sampled: {y_test_sampled.shape}")

Tamaño del conjunto y_train_sampled: (12000, 10)
Tamaño del conjunto y_test_sampled: (500, 10)


In [6]:
# 4. Transformación TF-IDF
tfidf = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf.fit_transform(X_train_sampled)
X_test_tfidf = tfidf.transform(X_test_sampled)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report, confusion_matrix
import os
import pandas as pd

# Crear el directorio de salida si no existe
output_dir = './outputs/classification/tfidf/'
os.makedirs(output_dir, exist_ok=True)

# 5. Definición de los modelos con OneVsRestClassifier
models = {
    "Logistic Regression": OneVsRestClassifier(LogisticRegression(random_state=42)),
    "Random Forest": OneVsRestClassifier(RandomForestClassifier(random_state=42)),
    "Support Vector Machine": OneVsRestClassifier(SVC(random_state=42)),
    "Gradient Boosting": OneVsRestClassifier(GradientBoostingClassifier(random_state=42))
}

# 6. Entrenamiento, predicción y evaluación de cada modelo
results = []
confusion_matrices = []
classification_reports = []

print(f"Tamaño del conjunto X_train_tfidf: {X_train_tfidf.shape}")
print(f"Tamaño del conjunto X_test_tfidf: {X_test_tfidf.shape}")
print(f"Tamaño del conjunto y_train_sampled: {y_train_sampled.shape}")
print(f"Tamaño del conjunto y_test_sampled: {y_test_sampled.shape}")

for model_name, model in models.items():
    print(f"Training model: {model_name}")
    
    # Entrenamiento
    model.fit(X_train_tfidf, y_train_sampled)
    
    print(f"Infering model: {model_name}")
    # Predicción
    y_pred = model.predict(X_test_tfidf)
    
    print(f"Shape y_test_sampled: {y_test_sampled.shape}")
    print(f"Shape y_pred: {y_pred.shape}")
    
    # Evaluación de métricas
    accuracy = accuracy_score(y_test_sampled, y_pred)
    precision = precision_score(y_test_sampled, y_pred, average='weighted')
    recall = recall_score(y_test_sampled, y_pred, average='weighted')
    f1 = f1_score(y_test_sampled, y_pred, average='weighted')
    
    # Agregar resultados al DataFrame
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    })

    # Generar y guardar la matriz de confusión (por cada etiqueta)
    cm = confusion_matrix(y_test_sampled.argmax(axis=1), y_pred.argmax(axis=1))
    confusion_matrices.append({"Model": model_name, "Confusion Matrix": cm})
    
    # Generar y almacenar el classification report
    report = classification_report(y_test_sampled, y_pred, target_names=mlb.classes_)
    classification_reports.append({"Model": model_name, "Classification Report": report})

# Convertir resultados a un DataFrame de pandas
results_df = pd.DataFrame(results)

# 7. Guardar resultados en un archivo CSV
results_df.to_csv(os.path.join(output_dir, 'model_comparison_results_6000.csv'), index=False)

# Guardar la matriz de confusión como un archivo CSV
confusion_matrices_df = pd.DataFrame(
    [(item['Model'], item['Confusion Matrix'].tolist()) for item in confusion_matrices],
    columns=['Model', 'Confusion Matrix']
)
confusion_matrices_df.to_csv(os.path.join(output_dir, 'confusion_matrices_6000.csv'), index=False)

# Guardar el classification report en un archivo de texto
with open(os.path.join(output_dir, 'classification_reports_6000.txt'), 'w') as file:
    for report in classification_reports:
        file.write(f"Model: {report['Model']}\n")
        file.write(f"{report['Classification Report']}\n")
        file.write("="*80 + "\n")

# 8. Mostrar resultados
print(results_df)

Tamaño del conjunto X_train_tfidf: (6000, 1000)
Tamaño del conjunto X_test_tfidf: (500, 1000)
Tamaño del conjunto y_train_sampled: (6000, 10)
Tamaño del conjunto y_test_sampled: (500, 10)
Training model: Logistic Regression
Infering model: Logistic Regression
Shape y_test_sampled: (500, 10)
Shape y_pred: (500, 10)
Training model: Random Forest
Infering model: Random Forest
Shape y_test_sampled: (500, 10)
Shape y_pred: (500, 10)
Training model: Support Vector Machine


Implementando Grid Search Cross Validation

In [7]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import os
import pandas as pd

# Crear el directorio de salida si no existe
output_dir = './outputs/classification/tfidf/'
os.makedirs(output_dir, exist_ok=True)

# 5. Definición de los modelos con OneVsRestClassifier
models = {
    "Logistic Regression": OneVsRestClassifier(LogisticRegression(random_state=42)),
    "Random Forest": OneVsRestClassifier(RandomForestClassifier(random_state=42)),
    "Support Vector Machine": OneVsRestClassifier(SVC(random_state=42)),
    "Gradient Boosting": OneVsRestClassifier(GradientBoostingClassifier(random_state=42))
}

# Definición de los hiperparámetros a probar con GridSearchCV para cada modelo
param_grids = {
    "Logistic Regression": {
        'estimator__C': [0.01, 0.1, 1, 10],  # Regularización
        'estimator__solver': ['lbfgs', 'liblinear']  # Solvers posibles
    },
    "Random Forest": {
        'estimator__n_estimators': [50, 100, 200],  # Número de árboles
        'estimator__max_depth': [None, 10, 20]  # Profundidad máxima de los árboles
    },
    "Support Vector Machine": {
        'estimator__C': [0.1, 1, 10],  # Regularización
        'estimator__kernel': ['linear', 'rbf']  # Tipo de kernel
    },
    "Gradient Boosting": {
        'estimator__n_estimators': [100, 200],  # Número de árboles
        'estimator__learning_rate': [0.01, 0.1, 0.2]
    }
}

# 6. Entrenamiento, predicción y evaluación de cada modelo con GridSearchCV
results = []
confusion_matrices = []
classification_reports = []

print(f"Tamaño del conjunto X_train_tfidf: {X_train_tfidf.shape}")
print(f"Tamaño del conjunto X_test_tfidf: {X_test_tfidf.shape}")
print(f"Tamaño del conjunto y_train_sampled: {y_train_sampled.shape}")
print(f"Tamaño del conjunto y_test_sampled: {y_test_sampled.shape}")

for model_name, model in models.items():
    print(f"Tuning model: {model_name}")
    
    # Configurar el GridSearchCV con verbosity para ver las combinaciones que se están probando
    grid_search = GridSearchCV(
        model, 
        param_grids[model_name], 
        scoring='f1_weighted', 
        cv=3, 
        n_jobs=-1, 
        verbose=2  # Nivel de verbose para imprimir el progreso de la búsqueda
    )
    
    # Entrenamiento con validación cruzada
    print(f"Starting GridSearchCV for {model_name}...")
    grid_search.fit(X_train_tfidf, y_train_sampled)
    
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    
    # Obtener el mejor modelo
    best_model = grid_search.best_estimator_
    
    print(f"Predicting with model: {model_name}")
    # Predicción
    y_pred = best_model.predict(X_test_tfidf)
    
    # Evaluación de métricas
    accuracy = accuracy_score(y_test_sampled, y_pred)
    precision = precision_score(y_test_sampled, y_pred, average='weighted')
    recall = recall_score(y_test_sampled, y_pred, average='weighted')
    f1 = f1_score(y_test_sampled, y_pred, average='weighted')
    
    # Agregar resultados al DataFrame
    results.append({
        "Model": model_name,
        "Best Parameters": grid_search.best_params_,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    })

    # Generar y guardar la matriz de confusión (por cada etiqueta)
    cm = confusion_matrix(y_test_sampled.argmax(axis=1), y_pred.argmax(axis=1))
    confusion_matrices.append({"Model": model_name, "Confusion Matrix": cm})
    
    # Generar y almacenar el classification report
    report = classification_report(y_test_sampled, y_pred, target_names=mlb.classes_)
    classification_reports.append({"Model": model_name, "Classification Report": report})

# Convertir resultados a un DataFrame de pandas
results_df = pd.DataFrame(results)

# 7. Guardar resultados en un archivo CSV
results_df.to_csv(os.path.join(output_dir, 'model_comparison_results_12000_CV.csv'), index=False)

# Guardar la matriz de confusión como un archivo CSV
confusion_matrices_df = pd.DataFrame(
    [(item['Model'], item['Confusion Matrix'].tolist()) for item in confusion_matrices],
    columns=['Model', 'Confusion Matrix']
)
confusion_matrices_df.to_csv(os.path.join(output_dir, 'confusion_matrices_12000_CV.csv'), index=False)

# Guardar el classification report en un archivo de texto
with open(os.path.join(output_dir, 'classification_reports_12000_CV.txt'), 'w') as file:
    for report in classification_reports:
        file.write(f"Model: {report['Model']}\n")
        file.write(f"{report['Classification Report']}\n")
        file.write("="*80 + "\n")

# 8. Mostrar resultados
print(results_df)

Tamaño del conjunto X_train_tfidf: (12000, 1000)
Tamaño del conjunto X_test_tfidf: (500, 1000)
Tamaño del conjunto y_train_sampled: (12000, 10)
Tamaño del conjunto y_test_sampled: (500, 10)
Tuning model: Logistic Regression
Starting GridSearchCV for Logistic Regression...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best parameters for Logistic Regression: {'estimator__C': 10, 'estimator__solver': 'liblinear'}
Predicting with model: Logistic Regression
Tuning model: Random Forest
Starting GridSearchCV for Random Forest...
Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .....estimator__C=0.01, estimator__solver=liblinear; total time=   4.2s
[CV] END .......estimator__C=10, estimator__solver=liblinear; total time=  11.5s
[CV] END estimator__max_depth=10, estimator__n_estimators=100; total time= 2.4min
[CV] END estimator__max_depth=20, estimator__n_estimators=50; total time= 2.0min
[CV] END .....estimator__C=0.01, estimator__solver=liblinear; total time

In [ ]:
import subprocess

# Configuración: nombre de la instancia y ubicación
instance_name = 'tfm-final'  # Nombre de la instancia que deseas detener
location = 'europe-west1-b'  # Zona de la instancia

# Comando de gcloud para detener la instancia
command = f"gcloud workbench instances stop {instance_name} --location={location}"

try:
    # Ejecutar el comando
    print(f"Deteniendo la instancia '{instance_name}' en la ubicación '{location}'...")
    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Mostrar salida
    print(result.stdout.decode('utf-8'))
except subprocess.CalledProcessError as e:
    print(f"Error al detener la instancia: {e.stderr.decode('utf-8')}")


Deteniendo la instancia 'tfm-final' en la ubicación 'europe-west1-b'...
